In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_article_content(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Вычленяем загаловок
    title_tag = soup.find('h1')
    title = title_tag.get_text(strip=True) if title_tag else 'No title'
    content = ''

    # Поиск тега с data-v-59f49686
    article_body = soup.find('article', {'data-v-59f49686': True})
    if article_body:
        paragraphs = article_body.find_all('p')
        content = ' '.join([p.get_text(strip=True) for p in paragraphs])

    # Если не сработало, ищем тег с data-v-68d8e5f5
    if not content:
        print(f"Content not found in {article_url} with data-v-59f49686, checking for data-v-68d8e5f5...")
        article_body_alt = soup.find('article', {'data-v-68d8e5f5': True})
        if article_body_alt:
            paragraphs = article_body_alt.find_all('p')
            content = ' '.join([p.get_text(strip=True) for p in paragraphs])
    if not content:
        content = 'No content available'

    return title, content

# Парсинг с главной страницы
def parse_lifehacker():
    base_url = 'https://lifehacker.ru/topics/technology/'
    articles = []

    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Первые 10 ссылок
    for article in soup.find_all('a', class_='lh-small-article-card__link')[:11]:
        article_url = 'https://lifehacker.ru' + article['href']
        title, content = get_article_content(article_url)

        articles.append({'Title': title, 'Content': content})  # Каждая строка будет словарем с заголовком и текстом

    return articles

articles_data = parse_lifehacker()
df = pd.DataFrame(articles_data)
styled_df = df.style.set_properties(**{
    'border': '2px solid black',
    'color': '#333333',
    'background-color': '#ffffff',
    'text-align': 'left',
    'padding': '8px'
}).set_table_styles([
    {
        'selector': 'thead th',
        'props': [
            ('background-color', '#4CAF50'),
            ('color', 'white'),
            ('border', '2px solid black')
        ]
    },
    {
        'selector': 'tbody tr:nth-child(even)',
        'props': [('background-color', '#f2f2f2')]
    },
    {
        'selector': 'tbody tr:nth-child(odd)',
        'props': [('background-color', '#ffffff')]
    }
])
styled_df